In [1]:
import numpy as np
import pandas as pd

In [489]:
companies =  pd.read_csv("companies.txt", sep=None, encoding="ISO-8859-1",engine='python')

print(companies.head())
print(companies.shape)
print(companies.info())
print(companies.isnull().sum());
companies.permalink= companies.permalink.str.encode('utf-8').str.decode('ascii', 'ignore')
companies.permalink = companies.permalink.str.upper()
print(companies.head())
print(companies.shape)
print(companies.info())
print(companies.isnull().sum());

# 66368 companies,66368 companies ,permalink,no,114949,16

                             permalink                    name  \
0                  /Organization/-Fame                   #fame   
1               /Organization/-Qounter                :Qounter   
2  /Organization/-The-One-Of-Them-Inc-  (THE) ONE of THEM,Inc.   
3                /Organization/0-6-Com                 0-6.com   
4       /Organization/004-Technologies        004 Technologies   

                        homepage_url  \
0                 http://livfame.com   
1             http://www.qounter.com   
2                http://oneofthem.jp   
3                 http://www.0-6.com   
4  http://004gmbh.de/en/004-interact   

                                       category_list     status country_code  \
0                                              Media  operating          IND   
1  Application Platforms|Real Time|Social Network...  operating          USA   
2                                  Apps|Games|Mobile  operating          NaN   
3                                        C

In [490]:
rounds2 =  pd.read_csv("rounds2.csv", sep=None, encoding="ISO-8859-1",engine='python')
rounds2.company_permalink= rounds2.company_permalink.str.encode('utf-8').str.decode('ascii', 'ignore').str.upper()
print(rounds2.head());
print(rounds2.info())

                     company_permalink  \
0                  /ORGANIZATION/-FAME   
1               /ORGANIZATION/-QOUNTER   
2               /ORGANIZATION/-QOUNTER   
3  /ORGANIZATION/-THE-ONE-OF-THEM-INC-   
4                /ORGANIZATION/0-6-COM   

                           funding_round_permalink funding_round_type  \
0  /funding-round/9a01d05418af9f794eebff7ace91f638            venture   
1  /funding-round/22dacff496eb7acb2b901dec1dfe5633            venture   
2  /funding-round/b44fbb94153f6cdef13083530bb48030               seed   
3  /funding-round/650b8f704416801069bb178a1418776b            venture   
4  /funding-round/5727accaeaa57461bd22a9bdd945382d            venture   

  funding_round_code   funded_at  raised_amount_usd  
0                  B  05-01-2015        10000000.00  
1                  A  14-10-2014                nan  
2                NaN  01-03-2014          700000.00  
3                  B  30-01-2014         3406878.00  
4                  A  19-03-2008      

In [491]:
#print(rounds2.head())
#print(rounds2.shape)
#print(rounds2.info())
#print(rounds2.isnull().sum());

rounds2_group_by = rounds2.groupby(by=['company_permalink'],as_index=False)
print(rounds2_group_by)

df = pd.DataFrame(rounds2_group_by.size()).reset_index()
print(df.info())
print(df.head(50))
print(df.shape)
output = companies.permalink.isin(df.company_permalink)  
print('The amount of companies present in companies and not in rounds2 are :- ', output.value_counts())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 2 columns):
company_permalink    66368 non-null object
0                    66368 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.0+ MB
None
                           company_permalink  0
0                        /ORGANIZATION/-FAME  1
1                     /ORGANIZATION/-QOUNTER  2
2        /ORGANIZATION/-THE-ONE-OF-THEM-INC-  1
3                      /ORGANIZATION/0-6-COM  1
4             /ORGANIZATION/004-TECHNOLOGIES  1
5           /ORGANIZATION/01GAMES-TECHNOLOGY  1
6        /ORGANIZATION/0NDINE-BIOMEDICAL-INC  2
7                       /ORGANIZATION/0XDATA  4
8                            /ORGANIZATION/1  3
9                  /ORGANIZATION/1-2-3-LISTO  1
10                     /ORGANIZATION/1-4-ALL  1
11            /ORGANIZATION/1-618-TECHNOLOGY  1
12               /ORGANIZATION/1-800-DENTIST  1
13               /ORGANIZATION/1-800-DOCTORS  1
14  /ORGANIZATION/1-800-PUBLICRE

In [537]:
master_frame = pd.merge(rounds2,companies,how='left', 
                        left_on=['company_permalink'], right_on=['permalink'])
print(master_frame.shape)
print(master_frame.info())
print(master_frame.head())
print(round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2))

(114949, 16)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 114949 entries, 0 to 114948
Data columns (total 16 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
permalink                  114949 non-null object
name                       114948 non-null object
homepage_url               108815 non-null object
category_list              111539 non-null object
status                     114949 non-null object
country_code               106271 non-null object
state_code                 104003 non-null object
region                     104782 non-null object
city                       104785 non-null object
founded_at                 94428 non-null object
dtypes: float64(1), object(15)
memory usage: 14.9+ MB
None
                   

In [538]:
master_frame.drop(['funding_round_code', 'founded_at', 'homepage_url', 'state_code', 'region', 'city'], axis=1, inplace=True)
print(round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2))


company_permalink          0.00
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd         17.39
permalink                  0.00
name                       0.00
category_list              2.97
status                     0.00
country_code               7.55
dtype: float64


In [539]:
print('No. of columns in master frame :',master_frame.shape[1])

# Check if any rows are completely null
print('No. of rows with all columns null in master frame:',len(master_frame[master_frame.isnull().sum(axis=1)==master_frame.shape[1]].index))

# Check if any row has mre than 3 columns null
print('No .of rows in master_frame with >3 columns as null :- ', len(master_frame[master_frame.isnull().sum(axis=1) > 3].index))

# Converting raised amount to mn as it is the required unit. We are talking about million dollars so any 
# calculationa beyond two decimal places of million dollars wont make much of a difference
master_frame['raised_amount_usd'] = master_frame['raised_amount_usd']/1000000

# Removing raised amount usd null values as we cannot impute these values as it will cause bias.
master_frame = master_frame[~np.isnan(master_frame['raised_amount_usd'])]
print(master_frame)
print(round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2))

# We have 6.16 % country code nulls, since this is a huge value and we cannot predict as it will cause bias. 
# So, the other option is to remove such rows.
master_frame = master_frame[master_frame.country_code.notnull()].reset_index()
print(master_frame.shape)
print(master_frame.head())
print(round(100*(master_frame.isnull().sum()/len(master_frame.index)), 2))



No. of columns in master frame : 10
No. of rows with all columns null in master frame: 0
No .of rows in master_frame with >3 columns as null :-  0
                                        company_permalink  \
0                                     /ORGANIZATION/-FAME   
2                                  /ORGANIZATION/-QOUNTER   
3                     /ORGANIZATION/-THE-ONE-OF-THEM-INC-   
4                                   /ORGANIZATION/0-6-COM   
6                        /ORGANIZATION/01GAMES-TECHNOLOGY   
7                     /ORGANIZATION/0NDINE-BIOMEDICAL-INC   
8                     /ORGANIZATION/0NDINE-BIOMEDICAL-INC   
9                                    /ORGANIZATION/0XDATA   
10                                   /ORGANIZATION/0XDATA   
11                                   /ORGANIZATION/0XDATA   
12                                   /ORGANIZATION/0XDATA   
13                                        /ORGANIZATION/1   
15                                        /ORGANIZATION/1   

(89108, 11)
   index                    company_permalink  \
0      0                  /ORGANIZATION/-FAME   
1      2               /ORGANIZATION/-QOUNTER   
2      4                /ORGANIZATION/0-6-COM   
3      6     /ORGANIZATION/01GAMES-TECHNOLOGY   
4      7  /ORGANIZATION/0NDINE-BIOMEDICAL-INC   

                           funding_round_permalink funding_round_type  \
0  /funding-round/9a01d05418af9f794eebff7ace91f638            venture   
1  /funding-round/b44fbb94153f6cdef13083530bb48030               seed   
2  /funding-round/5727accaeaa57461bd22a9bdd945382d            venture   
3  /funding-round/7d53696f2b4f607a2f2a8cbb83d01839        undisclosed   
4  /funding-round/2b9d3ac293d5cdccbecff5c8cb0f327d               seed   

    funded_at  raised_amount_usd                            permalink  \
0  05-01-2015              10.00                  /ORGANIZATION/-FAME   
1  01-03-2014               0.70               /ORGANIZATION/-QOUNTER   
2  19-03-2008               2.00   

In [540]:
master_frame_groupby_funding_type = master_frame[(master_frame['funding_round_type']=='seed') | (master_frame['funding_round_type']=='venture')| (master_frame['funding_round_type']=='angel') | (master_frame['funding_round_type']=='private_equity')]  .groupby(by=['funding_round_type'])
avg_funding_for_each_round = pd.DataFrame(master_frame_groupby_funding_type['raised_amount_usd'].mean()).reset_index()
pd.set_option('display.float_format', '{:.2f}'.format)
print(avg_funding_for_each_round)
fund_type_5_to_15_million = avg_funding_for_each_round.funding_round_type[ (avg_funding_for_each_round.raised_amount_usd>5) & ((avg_funding_for_each_round.raised_amount_usd<15))]
print('The suitable funding round for investment would be :',fund_type_5_to_15_million)
# venture 11.75 mnUSD, seed - 0.72mn , angel - 0.96 mn, private_equity - 73.31 mn usd , venture

  funding_round_type  raised_amount_usd
0              angel               0.97
1     private_equity              73.62
2               seed               0.75
3            venture              11.74
The suitable funding round for investment would be : 3    venture
Name: funding_round_type, dtype: object


In [559]:
venture_data_frame = master_frame.loc[master_frame['funding_round_type'].isin(['venture']),:]
venture_data_frame.sort_values(by=['raised_amount_usd'],ascending=False,inplace=True)
countries_with_highest_funding.drop_duplicates(keep='first',inplace=True);
print(type(countries_with_highest_funding))
top9 = countries_with_highest_funding[:9]
print(top9)
# USA ,India, GBR    

<class 'pandas.core.series.Series'>
29439    USA
86978    CHN
28538    IND
71523    GBR
21210    NLD
73059    SWE
77877    FRA
32182    SGP
67125    NOR
Name: country_code, dtype: object


/Users/karandeepmalik/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [595]:
sector_mapping =  pd.read_csv("mapping.csv", encoding="ASCII") 
print(sector_mapping.head())
print(sector_mapping.info())
print(sector_mapping.shape)
category_main_sector_mapping = pd.DataFrame()
category_main_sector_mapping['primary sector'] =sector_mapping['category_list']
category_main_sector_mapping['main sector'] = sector_mapping.apply(lambda x: x.isin([1]), axis=1).apply(lambda x: list(sector_mapping.columns[x]), axis=1)
print(category_main_sector_mapping.head(50))

print(master_frame.shape)
primary_sector_master = master_frame['category_list'].apply(lambda x: (x.split('|')[0]) if pd.notnull(x) else x).reset_index(name='primary sector')

print(primary_sector_master)

   category_list  Automotive & Sports  Blanks  Cleantech / Semiconductors  \
0            NaN                    0       1                           0   
1             3D                    0       0                           0   
2    3D Printing                    0       0                           0   
3  3D Technology                    0       0                           0   
4     Accounting                    0       0                           0   

   Entertainment  Health  Manufacturing  News, Search and Messaging  Others  \
0              0       0              0                           0       0   
1              0       0              1                           0       0   
2              0       0              1                           0       0   
3              0       0              1                           0       0   
4              0       0              0                           0       0   

   Social, Finance, Analytics, Advertising  
0                

In [604]:

combined_frame = pd.concat( [master_frame,primary_sector_master[['primary sector']]],axis=1) 
print(combined_frame.shape)
#print(combined_master_primary_sector_main_sector.head())
combined_frame = combined_frame.merge(category_main_sector_mapping,
                                                      on='primary sector',how='left')
print(combined_frame)
combined_frame['main sector']=combined_frame['main sector'].fillna("")
group_by_main_sector = combined_frame.groupby(by=['main sector'])
combined_frame['Main Sector Inv Count'] =group_by_main_sector['raised_amount_usd'].count()



(89108, 12)
        index                                  company_permalink  \
0           0                                /ORGANIZATION/-FAME   
1           2                             /ORGANIZATION/-QOUNTER   
2           4                              /ORGANIZATION/0-6-COM   
3           6                   /ORGANIZATION/01GAMES-TECHNOLOGY   
4           7                /ORGANIZATION/0NDINE-BIOMEDICAL-INC   
5           8                /ORGANIZATION/0NDINE-BIOMEDICAL-INC   
6           9                               /ORGANIZATION/0XDATA   
7          10                               /ORGANIZATION/0XDATA   
8          11                               /ORGANIZATION/0XDATA   
9          12                               /ORGANIZATION/0XDATA   
10         13                                    /ORGANIZATION/1   
11         15                                    /ORGANIZATION/1   
12         16                          /ORGANIZATION/1-2-3-LISTO   
13         20                       

TypeError: unhashable type: 'list'

                              company_permalink  \
37307  /ORGANIZATION/1-800-PUBLICRELATIONS-INC-   
640                  /ORGANIZATION/1-MAINSTREAM   
58748              /ORGANIZATION/128-TECHNOLOGY   
59326           /ORGANIZATION/1366-TECHNOLOGIES   
59325           /ORGANIZATION/1366-TECHNOLOGIES   

                               funding_round_permalink funding_round_type  \
37307  /funding-round/11c228f58831bc7ed337ef69ecc560c2     private_equity   
640    /funding-round/b952cbaf401f310927430c97b68162ea            venture   
58748  /funding-round/fb6216a30cb566ede89e0bee0623a634            venture   
59326  /funding-round/6d3f3797371956ece035b8478c1441b2            venture   
59325  /funding-round/424129ce1235cfab2655ee81305f7c2b            venture   

        funded_at  raised_amount_usd  \
37307  01-02-2015         6000000.00   
640    17-03-2015         5000000.00   
58748  16-12-2014        11999347.00   
59326  09-04-2015         5000000.00   
59325  15-10-2013        15000

TypeError: unhashable type: 'list'